# Simple WordCount Example for MapReduce using Hadoop Streaming and Python

By: Vahid Mostofi

### ceate some input files

In [1]:
!mkdir data
!echo "dog cat cat apple ninja" > data/text1.txt
!echo "ninja apple cat keyboard horse" > data/another_file.txt
!echo "water cat dog spider spider" > data/yet_another_file.txt

### move input files to hdfs

In [1]:
!hadoop fs -mkdir -p /outputs
!hadoop fs -mkdir -p /inputs
!hadoop fs -put data /inputs

put: `/inputs/data/text1.txt': File exists
put: `/inputs/data/another_file.txt': File exists
put: `/inputs/data/yet_another_file.txt': File exists


### list the content of the HDFS folder we just created
The three files we created are now stored on the HDFS

In [2]:
!hadoop dfs -ls /inputs/data


Found 3 items
-rw-r--r--   3 root supergroup         31 2020-08-25 20:58 /inputs/data/another_file.txt
-rw-r--r--   3 root supergroup         24 2020-08-25 20:58 /inputs/data/text1.txt
-rw-r--r--   3 root supergroup         28 2020-08-25 20:58 /inputs/data/yet_another_file.txt


### mapper
this is the word count example, so we need to create the mapper and reducer.

 * the ```%%writefile mapper.py``` tells Jupyter to save the content of the cell as a file named mapper.py in the same direcotry. So the #!/opt/bit.... is the first line of the file.
 * the ```#!/opt/bitnami/python/bin/python``` specifies the Python path for running the file.
 * the mapper reads from the stdin, which is provided to it by the MapReduce framework and also writes to stdout (using print).

In [3]:
%%writefile mapper.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*
import sys
for line in sys.stdin: # reads from stdin
    print("your message B", file=sys.stderr)
    line = line.strip()
    words = line.split()

    for word in words: # writes to stdout
        print("%s\t%d" % (word, 1))

Overwriting mapper.py


### reducer
similarly to previous cell, here we create a file, named ```reducer.py``` and store the logic for our reducer.

In [4]:
%%writefile reducer.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*

import sys
total = 0
lastword = None

for line in sys.stdin:
    line = line.strip()

    # recuperer la cle et la valeur et conversion de la valeur en int
    word, count = line.split()
    count = int(count)

    # passage au mot suivant (plusieurs cles possibles pour une même exécution de programme)
    if lastword is None:
        lastword = word
    if word == lastword:
        total += count
    else:
        print("%s\t%d occurences" % (lastword, total))
        total = count
        lastword = word

if lastword is not None:
    print("%s\t%d occurences" % (lastword, total))

Overwriting reducer.py


### run MapReduce
now to we need to run the map reduce program using our mapper.py and reducer.py

 * ```!hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar ``` specifies the path for streaming module of MapReduce
 * ```-file``` tells map-reduce which files should be moved to the worker nodes, you can also move txt files and read them in your mapper.py or reducer.py
 * ```-mapper``` and ```-reducer``` specify the the commands for mapper and reducer, because mapper.py file has the path to python as the first line, the system would know how to execute it.
 * ```-input``` tells which folder should be scanned for input, all the files in this folder would be fed to the mappers (mapper.py) as stdin
 * ```-output``` specifies the path for the folder which the output of the map-reduce execution should be stored. Remmeber the folder must be empty, in other words every single execution of the following command needs a new folder. You don't need to crate the folder before hand, just make sure you use a new path each time.
 
 * for more infomration about map-reduce streaming API please use  http://hadoop.apache.org/docs/r1.2.1/streaming.html


In [7]:
!hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar \
    -file $PWD/mapper.py\
    -file $PWD/reducer.py\
    -mapper mapper.py \
    -reducer reducer.py \
    -input /inputs/data \
    -output /outputs/result2

2020-08-25 21:33:39,826 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/training/mapper.py, /training/reducer.py, /tmp/hadoop-unjar5501641474586690589/] [] /tmp/streamjob7630392549135838828.jar tmpDir=null
2020-08-25 21:33:41,205 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.19.0.4:8032
2020-08-25 21:33:41,489 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.19.0.2:10200
2020-08-25 21:33:41,525 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.19.0.4:8032
2020-08-25 21:33:41,526 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.19.0.2:10200
2020-08-25 21:33:41,858 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1598390533934_0002
2020-08-25 21:33:42,058 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = fals

### lets look at the output of the map reduce

In [8]:
!hdfs dfs -ls /outputs/result2

Found 2 items
-rw-r--r--   3 root supergroup          0 2020-08-25 21:34 /outputs/result2/_SUCCESS
-rw-r--r--   3 root supergroup        152 2020-08-25 21:34 /outputs/result2/part-00000


In [9]:
!hdfs dfs -cat /outputs/result2/part-00000

2020-08-25 21:36:33,512 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
apple	2 occurences
cat	4 occurences
dog	2 occurences
horse	1 occurences
keyboard	1 occurences
ninja	2 occurences
spider	2 occurences
water	1 occurences


### make a directory out of HDFS and store the results

In [10]:
!mkdir -p /outputs/res_out_of_hdfs
!hdfs dfs -copyToLocal /outputs/result2/* /outputs/res_out_of_hdfs

2020-08-25 21:38:54,446 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


In [11]:
!ls /outputs/res_out_of_hdfs

_SUCCESS  part-00000


In [13]:
!cat /outputs/res_out_of_hdfs/part-00000

apple	2 occurences
cat	4 occurences
dog	2 occurences
horse	1 occurences
keyboard	1 occurences
ninja	2 occurences
spider	2 occurences
water	1 occurences
